In [ ]:
import os
import sys
import requests
from bs4 import BeautifulSoup

directory_path = os.getcwd()

relevant_part = directory_path.split("scripts")[0]

# Use os.path.dirname() para subir um nível no diretório
parent_directory = os.path.dirname(relevant_part.rstrip(os.sep))


In [ ]:

sys.path.insert(0, parent_directory)

from scripts.load import Loader

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/nayanegomes/AuctionHouseMonitor/auction-house-monitor-85951197a9c3.json"

In [ ]:
import pandas as pd

# Configurações para mostrar mais linhas
pd.set_option('display.max_rows', None)  # Mostra todas as linhas

# Configurações para mostrar mais colunas
pd.set_option('display.max_columns', None)  # Mostra todas as colunas

# Opcionalmente, ajustar a largura do display para evitar quebra de linha nas células
pd.set_option('display.width', 1000)

# Ajustar a precisão da exibição para números de ponto flutuante, se necessário
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
loader = Loader(config_path='../config/config.yaml')

In [ ]:

# Exemplo de uso do novo método
df = loader.read_csv_from_gcp('data/integrated/Lista_imoveis_MG.csv')

In [ ]:
df = df.head(1)

In [ ]:
for index, row in df.iterrows():
    url = row['Link de acesso']

    # Adicionando cabeçalhos HTTP para simular um navegador
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

    
    # Fazendo a solicitação HTTP para obter o conteúdo do site
    response = requests.get(url, headers=headers)
    
    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Parsing do conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Buscando todos os elementos com a classe 'related-box'
        related_boxes = soup.find_all('div', class_='related-box')
        
        if related_boxes:
            # Iterando sobre os elementos encontrados e armazenando o texto que vem após "Descricao"
            descricao_encontrada = False
            for i, box in enumerate(related_boxes, start=1):
                texto = box.get_text(separator='\n', strip=True)
                
                if 'Descrição:' in texto:
                    descricao_encontrada = True
                    # Pegando o texto após "Descricao"
                    descricao = texto.split('Descrição:', 1)[1].strip()
                    df.at[index, 'Descricao'] = descricao
                    break  # Saindo do loop após encontrar a primeira ocorrência
            if not descricao_encontrada:
                print(f'Nenhuma "Descrição:" encontrada nas divs com a classe "related-box" para o link {url}')
        else:
            print(f'Nenhuma div com a classe "related-box" foi encontrada para o link {url}')
    else:
        print(f'Erro ao acessar o site: {response.status_code} para o link {url}')
